In [1]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

In [11]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)

In [3]:
import json 
with open('document.json') as f_in: 
    docs_raw = json.load(f_in)
documents = []
for course_dict in docs_raw : 
    for doc in course_dict['documents'] : 
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

/home/nhutpham/anaconda3/envs/N2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
#created the dense vector using the pre-trained model
operations = []
for i, doc in enumerate(documents):
    doc["id"] = i
    doc["vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [17]:
print("Data has", len(documents), "entities, each with fields: ", documents[0].keys())
print("Vector dim:", len(documents[0]["vector"]))

Data has 948 entities, each with fields:  dict_keys(['text', 'section', 'question', 'course', 'text_vector', 'vector', 'id'])
Vector dim: 768


In [18]:
res = client.insert(collection_name="demo_collection", data=documents)

print(res)

{'insert_count': 948, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [22]:
query_vectors = [model.encode("Windows or Mac?")]

res = client.search(
    collection_name="demo_collection",  
    data=query_vectors,  
    limit=5,  
    output_fields=["text"],  
)

print(res)

data: ['[{\'id\': 35, \'distance\': 0.42958375811576843, \'entity\': {\'text\': \'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully\'}}, {\'id\': 822, \'distance\': 0.22694724798202515, \'entity\': {\'text\': \'If you wish to use WSL on your windows machine, here are the setup instructions:\\nCommand: Sudo apt install wget\\nGet Anaconda download address here. wget <download address>\\nTurn on Docker Desktop WFree Download | AnacondaSL2\\nCommand: git clone <github repository address>\\nVSCODE on WSL\\nJupyter: pip3 install jupyter\\nAdded by Gregory Morris (gwm1980@gmail.com)\\nAll in all softwares at one shop:\\nYou can use anaconda which has all built in services like pycharm, jupyter\\nAdded by Khaja Zaffer (khajazaffer@aln.iseg.ulisboa.pt)\\nFor windows “wsl --install” in Powershell\\nAdded by Vadim Surin (vdmsurin@gmai.com)\'}}, {\'id\': 503, \'distance\': 0.21111181378364563, \'entity\': {\'text\': \'That’s normal. We all h

In [28]:
res[0][0]['entity']

{'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}